# Agenda: Day 2

1. Recap and Q&A
2. Dtypes and `NaN`
3. Data frames (2D data)
    - Creating data frames
    - Retrieving rows
    - Retrieving columns
    - Naming the index and the columns
4. Adding and removing data
5. Useful methods and attributes
6. Boolean indexes
7. Querying with `.loc`
    - Row selectors
    - Column selectors
    - Assigning via `.loc`
8. Reading CSV data

# Recap

- Pandas is for reading, writing, manipulating, cleaning, and analyzing data
- Last time, we talked about the *Series*
- A series contains a bunch of values, all of the same type
- Retrieve from a series using `.loc` (by index) or `.iloc` (by position)
- We can set the index either when we create the series or assign a new value
- We can retrieve using a mask index via a boolean series
- Most operations performed on two series happen via the index
- If we have a series and a scalar value, the operation is "broadcast" to every element of the series

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame